In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random 
import queue
import time
from __init__ import pqdict #bibliothèque implémentant une file de priorité sous forme de dictionnaire
                            #nécessaire pour la task 4.

# Fonctions auxiliaires

In [2]:
"""Deux fonctions auxiliaires de calcul de distance"""

def distance(i,j):
    #retourne la distance (euclidienne) du point i au point j.

    d_2 = (i[0]-j[0])**2 + (i[1]-j[1])**2
    return np.sqrt(d_2)

def distanceE(i,P):
    #Retourne la distance du point i à l'ensemble de point P, ainsi qu'un point de P réalisant la distance.
    #Complexité en O(len(P))

    d = distance(i,P[0])
    pA = P[0]
    for point in P:
        if d > distance(i,point):
            d = distance(i,point)
            pA = point
    
    return d, pA

# Quatre algorithmes de subsampling

## Tâche 1 : L'algorithme naïf

In [3]:
def Task1(P,p,k):
    #Complexité en O(n^3)
    
    S = [P.pop(P.index(p))] #Initialisation de la liste des points issus du sample (en O(n))
    
    for i in range(k-1): #k-1 itérations
        dMax = 0
        targetP = P[0]
        
        for point in P: #on cherche le point le plus loin de S, au plus n itérations
            
            d, pA = distanceE(point,S) #O(k) car len(S) <= k
            if dMax < d:
                dMax = d
                targetP = point
                
        S.append(targetP)
        P.pop(P.index(targetP)) 
    
    return S

## Tâche 2 : Un raffinement quadratique

In [4]:
def Task2(P,p,k):
    S = [P.pop(P.index(p))] #O(n)
    
    distP = {point: distanceE(point,S)[0] for point in P} 
    #Ce dictionnaire stockera les distances des points de P à S
    
    
    for i in range(k-1): #k-1 itérations pour obtenir un sample de k points
        dMax = 0
        targetP = P[0]
        
        for point in P: #On trouve le point le plus de S dans P en cherchant le maximum dans distP
            if dMax < distP[point]:
                dMax = distP[point]
                targetP = point
                
        S.append(targetP) #On ajoute à S le point trouvé
        P.pop(P.index(targetP)) #On le supprime de P
        
        for point in P: #On met à jour les entrées de distP
            distP[point] = min(distP[point],distance(point,targetP))
            
    return S

## Tâche 3 : Une optimisation pour les $k$ faibles utilisant le contexte géométrique

In [5]:
def Task3(P,p,k):

    S = [P.pop(P.index(p))] #O(n)
    
    qP = queue.Queue() 
    #On implémente la données d'une région associée à un centre donnée par une fille FIFO
    
    #On commence par trouver le rayon du premier centre et un point réalisant cette distance
    d = 0 
    pA = S[0]
    for p in P:
        qP.put(p)
        if distance(p,S[0]) >= d:
            d = distance(p,S[0])
            pA = p
    
    regionS = {S[0]: (d, pA, qP)} 
    #Pour un centre s donné, on stocke les points de la région de s dans un dictionnaire ayant pour clé s et pour
    #valeur associée le rayon du centre, un point de la région le réalisant et la liste des points de la région
    
    
    for i in range(k-1): #k-1 itérations pour obtenir un sample de k points
        
        rmax = 0
        center = (0,0)
        
        for c in regionS: #On trouve le centre de rayon maximum en O(k) operations
            if regionS[c][0] >= rmax:
                rmax = regionS[c][0]
                center = c

        newC = regionS[center][1] #newC est le prochain centre
        
        #On met maintenant à jour la strucure regionS
        
        dC = 0 #On initialise le rayon de newC
        pC = newC #On initialise le point de la region de newC réalisant le rayon
        lC = queue.Queue() #On initialise la file des points de la region de newC
        
        for c in regionS:
            if distance(newC,c) <= 2*regionS[c][0]: #Seuls les points vérifiant cette condition sont suceptibles
                                                    #de se faire voler des points de leur région par newC
                                                    #(c.f. rapport)
                
                #On initialise les nouvelles variables associées à c
                rMajC = 0 #nouveau rayon de c
                pAMajC = c #nouveau point de la region de c réalisant le rayon
                lMajC = queue.Queue() #nouvelle file de la région de c
                
                for i in range(regionS[c][2].qsize()): #On met à jour les variables associées au centre c

                    p = regionS[c][2].get()

                    if distance(newC,p) <= distance(c,p): #On regarde si le point de la région de c est volé par
                                                          #newC ou non
                        lC.put(p)

                        if distance(p,newC) > dC: #Si cela devient un point de la région de newC, on met à jour
                                                  #les variables de newC
                            dC = distance(p,newC)
                            pC = p
                    else:                         #autrement on met à jour les variables associées à c
                        lMajC.put(p)
                        if distance(p,c) >= rMajC:
                            pAMajC = p
                            rMajC = distance(p,c)

                regionS[c] = (rMajC,pAMajC,lMajC) #On injecte dans la datastructure les nouvelles variables 
                                                  #associées à c

        regionS[newC] = (dC,pC,lC) #On injecte dans la datastructure les nouvelles variables associées à newC
        
        S.append(newC)

    return S

## Tâche 4 : une optimisation plus générale

In [6]:
def Task4(P,p,k):
    S = [P.pop(P.index(p))] #O(n)
    
    qP = queue.Queue() #On implémente la données d'une région associée à un centre donnée par une fille FIFO
    
    #On commence par trouver le rayon du premier centre et un point réalisant cette distance
    d = 0 
    pA = S[0]
    for p in P:
        qP.put(p)
        if distance(p,S[0]) > d:
            d = distance(p,S[0])
            pA = p
    
    regionS = {S[0]: (d, pA, qP)}
    #Pour un centre s donné, on stocke les points de la région de s dans un dictionnaire ayant pour clé s et pour
    #valeur associée le rayon du centre, un point de la région le réalisant et la liste des points de la région
    
        
    prio = pqdict() #On initialise une file de priorité permettant d'acceder au centre de plus grand rayon 
                    #en temps constant
    prio.additem(S[0],-d) #-d et non d car l'élément priorité est celui avec la clé de priorité la plus faible 
    
    Gf = {S[0] : [S[0]]} #On initialise le graphe des amis, implémenté sous la forme d'une
                         # liste de voisinage (le graphe est non orienté à priori)
        
    for i in range(k-1): #k-1 itérations pour obtenir un sample de k points
        
        center = prio.top() #On récupère le centre de plus grand rayon en O(1) par la file de priorité
        newC = regionS[center][1] #newC est le prochain centre
        
        #On met maintenant à jour la strucure regionS
        
        dC = 0 #On initialise le rayon de newC
        pC = newC #On initialise le point de la region de newC réalisant le rayon
        lC = queue.Queue() #On initialise la file des points de la region de newC
        
        f = []
        rCenter = regionS[center][0]
        for c in Gf[center]: #Il suffit de mettre à jour les régions des points qui sont amis avec le centre
                          #d'origine de newC
            if distance(center,c) <= 2*regionS[c][0] + rCenter:
                f.append(c)
                
                #On initialise les nouvelles variables associées à c
                rMajC = 0 #nouveau rayon de c
                pAMajC = c #nouveau point de la region de c réalisant le rayon
                lMajC = queue.Queue() #nouvelle file de la région de c

                for i in range(regionS[c][2].qsize()): #On met à jour les variables associées au centre c

                    p = regionS[c][2].get()

                    if distance(newC,p) <= distance(c,p): #On regarde si le point de la région de c est volé par
                                                          #newC ou non
                        lC.put(p)

                        if distance(p,newC) > dC: 
                            dC = distance(p,newC)
                            pC = p
                    else:                         #autrement on met à jour les variables associées à c
                        lMajC.put(p)
                        if distance(p,c) >= rMajC:
                            pAMajC = p
                            rMajC = distance(p,c)

                regionS[c] = (rMajC,pAMajC,lMajC) #On injecte dans la datastructure les nouvelles variables 
                                                  #associées à c
                prio.updateitem(c,-rMajC) #We update the radius of the previous centers in the priority queue
                

        regionS[newC] = (dC,pC,lC) #On injecte dans la datastructure les nouvelles variables associées à newC
        
        prio.additem(newC,-dC) #On ajoute à la file de priorité le nouveau centre newC
        S.append(newC)
        Gf[newC] = []
        
        Gf[center] = f.copy() #On met à jour la liste des amis de center
        #On ajoute newC au graphe des amis
        
        #On met maintenant à jour le graphe des amis 
        #On sait que seul newC, center, et les amis de center ont besoin d'être mis à jour (c.f. rapport)
        
        for c in Gf: #On met à jour le graphe 
            if distance(newC,c) <= 2*regionS[c][0] + regionS[newC][0]: #On regarde qui sont les voisins de newC
                Gf[newC].append(c)
            if distance(newC,c) <= 2*regionS[newC][0] + regionS[c][0] and c != newC: #On regarde qui a pour voisin newC
                Gf[c].append(newC)
    
            
    return S